<a href="https://colab.research.google.com/github/JohnPaul-del/med_project/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import cv2
import pandas as pd
import numpy as np
import pydicom
import matplotlib.pylab as plt


from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K
from tensorflow_addons.optimizers import RectifiedAdam as RAdam
from tqdm import tqdm
from PIL import Image


DICOM_DIR = 'predict/'
HEMM_TYPES = ['any', 'epidural', 'intraparenchymal',
               'intraventricular', 'subarachnoid', 'subdural']
DICOM_OBJECTS = os.listdir(DICOM_DIR)

In [ ]:
def window_image(img, center, width, intercept, slope, rescale=True):

    """
    Scales the input image by multiplying each pixel value by slope and adding intercept
    """

    img = (img * slope + intercept)
    img_min = center - width // 2
    img_max = center + width // 2
    img[img < img_min] = img_min
    img[img > img_max] = img_max

    if rescale:
        img = (img - img_min) / (img_max - img_min)

    return img


def get_dicom_fields(dcm_obj):
    """
    Get data from from DICOM object
    :param: dcm_obj
    :return: value of field

    """

    if type(dcm_obj) == pydicom.multival.MultiValue:
        return int(dcm_obj[0])
    else:
        return int(dcm_obj)


def get_sizes(dcm_obj):
    """
    Get values of image size
    :param: dcm_obj
    :return: image`s size
    """

    dicom_fields = [dcm_obj[('0028','1050')].value,
                    dcm_obj[('0028','1051')].value,
                    dcm_obj[('0028','1052')].value,
                    dcm_obj[('0028','1053')].value]

    return [get_dicom_fields(el) for el in dicom_fields]



def img_from_dicom(dicom_file):

    """
    Create image in png format from DICOM format

    :param dicom_file: DICOM file
    :return: image in png format
    """

    dcm = pydicom.read_file(dicom_file)
    window_center, window_width, intercept, slope = get_sizes(dcm)
    img = dcm.pixel_array
    img = window_image(img, window_center, window_width, intercept, slope)
    resized = cv2.resize(img, (224, 224))

    return resized


def create_test_gen(df, directory, x_col, size, batch_size):

    """
    Creating generator fro prediction
    Returns: Image Generator
    """

    return ImageDataGenerator().flow_from_dataframe(
        df,
        directory=directory,
        x_col=x_col,
        class_mode=None,
        target_size=(size, size),
        batch_size=batch_size,
        shuffle=True
    )


def resize_dcm(dcm_files, load_dir):

    """
    Resize and convert DICOM objcts to PNG
    """

    save_dir = 'tmp/'
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    for file_name in tqdm(dcm_files):
        try:
            path = load_dir + file_name
            new_path = save_dir + file_name.replace('.dcm', '.png')
            dcm = pydicom.dcmread(path)

            window_center, window_width, intercept, slope = get_sizes(dcm)
            img = dcm.pixel_array
            img = window_image(img, window_center, window_width, intercept, slope)
            resized = cv2.resize(img, (224, 224))
            cv2.imwrite(new_path, resized)

        except ValueError:
            continue


def bse_dice_loss(y_true, y_pred, smooth=1e-6):

  y_true = K.cast(y_true, 'float32')
  y_pred = K.cast(y_pred, 'float32')

  inputs = K.flatten(y_pred)
  targets = K.flatten(y_true)

  binary_loss = binary_crossentropy(targets, inputs)
  intersection = inputs * targets

  dice_loss = 1 - (2 * intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)

  dice_bce = binary_loss + dice_loss

  return dice_bce

In [ ]:
# Load model

model = load_model('rsna_mod_tf2_effnet_b5.h5', custom_objects={'bse_dice_loss': bse_dice_loss})

In [ ]:
# Convert DICOM files and save to tmp directory for prediction.
# Create a dataframe with filenames

tmp_res = os.listdir('tmp/')
df = pd.DataFrame(tmp_res, columns=['ID'])
images = DICOM_OBJECTS
resize_dcm(images, DICOM_DIR)

,ID
0,ID_14179b55c.png
1,ID_ad90a7582.png


In [ ]:
# Create generator object for prediction


test_gen = create_test_gen(
    df,
    directory='tmp/',
    x_col='ID',
    size=224,
    batch_size=2,
)


y_test = model.predict(
    test_gen,
    steps=len(test_gen),
    verbose=1
)

# Concatenate two dataframes and show prediction results

res_df = df.join(
    pd.DataFrame(
        y_test,
        columns=['any', 'epidural', 'intraparenchymal',
               'intraventricular', 'subarachnoid', 'subdural']
    )
)

res_df



1/1 [==============================] - 0s 76ms/step


,ID,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_14179b55c.png,0.133463,0.004151,0.025392,0.039878,0.058431,0.022560
1,ID_ad90a7582.png,0.010063,0.000525,0.001766,0.000474,0.003406,0.004358


In [ ]:
shutil.rmtree('tmp')